# Tutorial: Tradução de Textos com LangChain e Groq
Este notebook demonstra como utilizar a biblioteca LangChain, em conjunto com o provedor Groq, para realizar traduções automáticas de textos utilizando modelos de linguagem avançados (como Llama 3).

Você aprenderá a:
- Instalar e configurar o ambiente necessário;
- Utilizar prompts dinâmicos para tradução;
- Enviar mensagens para o modelo e interpretar as respostas.

Cada célula de código contém comentários explicativos para facilitar o entendimento.

In [ ]:
# Instala o pacote langchain com suporte ao provedor Groq.
# Necessário para utilizar modelos de linguagem hospedados na Groq, como o Llama 3.
pip install -qU "langchain[groq]"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Instalação dos Pacotes Necessários
Antes de começar, é preciso instalar o pacote `langchain[groq]`, que permite a integração com modelos de linguagem hospedados na Groq.

In [ ]:
# Importa bibliotecas para manipulação de variáveis de ambiente e entrada de usuário
import getpass
import os

# Solicita a chave da API Groq caso ainda não esteja definida na variável de ambiente.
# Isso é necessário para autenticar e acessar os modelos da Groq.
if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"]= getpass.getpass("Entre com a chave da API Groq: ")

# Importa função para inicializar o modelo de chat da LangChain
from langchain.chat_models import init_chat_model

# Inicializa o modelo Llama 3 hospedado na Groq para uso em tarefas de linguagem natural.
model = init_chat_model("llama3-8b-8192",model_provider="groq")

## Configuração da Chave de API e do Modelo
Para utilizar os modelos da Groq, é necessário informar sua chave de API. Em seguida, inicializamos o modelo Llama 3, que será usado para as traduções.

In [ ]:
# Importa classes para criar mensagens do sistema e do usuário (humano)
from langchain_core.messages import HumanMessage, SystemMessage

# Cria uma lista de mensagens para o modelo:
# - Mensagem do sistema define a tarefa (traduzir para espanhol)
# - Mensagem do humano contém o texto a ser traduzido
messages = [
    SystemMessage("Traduza as palavras a seguir para o espanhol:"),
    HumanMessage("Olá, como você está?"),
# A lista pode ser expandida com mais mensagens se necessário
]

# Envia as mensagens para o modelo e retorna a resposta traduzida
model.invoke(messages)

AIMessage(content='¡Hola! ¿Cómo estás?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 35, 'total_tokens': 44, 'completion_time': 0.007350129, 'prompt_time': 0.004475604, 'queue_time': 0.14597242500000002, 'total_time': 0.011825733}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_5b339000ab', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--03f55d84-ca0d-4ddf-b56c-8d3caeb125a8-0', usage_metadata={'input_tokens': 35, 'output_tokens': 9, 'total_tokens': 44})

## Criando Mensagens para Tradução
Agora vamos criar as mensagens que serão enviadas ao modelo. A mensagem do sistema define a tarefa (traduzir para um idioma específico) e a mensagem do humano contém o texto a ser traduzido.

In [ ]:
# Exemplo de streaming: recebe a resposta do modelo token por token.
# Útil para visualizar a resposta em tempo real, especialmente para respostas longas.
for token in model.stream(messages):
    print(token.content, end="|")  # Imprime cada token seguido de '|'

|¡|Hola|!| ¿|Cómo| est|ás|?||

## Streaming da Resposta do Modelo
Também é possível receber a resposta do modelo em tempo real, token por token. Isso é útil para acompanhar respostas longas à medida que são geradas.

In [ ]:
# Importa a classe para criar prompts dinâmicos para chat com modelos de linguagem
from langchain_core.prompts import ChatPromptTemplate

# Define o template do sistema, permitindo parametrizar o idioma de destino na tradução.
system_template = "Traduza as palavras a seguir para o {idioma}"

# Cria o template do prompt, combinando mensagens do sistema e do usuário.
# Isso permite reutilizar a estrutura para diferentes idiomas e textos.
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),  # Mensagem do sistema com template para idioma
    ("human", "{texto}"),        # Mensagem do humano com o texto a ser traduzido
])

# Gera o prompt preenchendo os parâmetros: idioma e texto desejados.
prompt = prompt_template.invoke({"idioma": "francês", "texto": "Olá, como você está?"})
prompt  # Exibe o prompt gerado para conferência

ChatPromptValue(messages=[SystemMessage(content='Traduza as palavras a seguir para o francês', additional_kwargs={}, response_metadata={}), HumanMessage(content='Olá, como você está?', additional_kwargs={}, response_metadata={})])

## Criando Prompts Dinâmicos para Tradução
Utilizando o `ChatPromptTemplate`, é possível criar prompts dinâmicos e reutilizáveis, facilitando a tradução para diferentes idiomas e textos.

In [ ]:
# Importa a classe para criar prompts dinâmicos para chat
from langchain_core.prompts import ChatPromptTemplate

# Define o template do sistema para tradução parametrizada
system_template = "Traduza as palavras a seguir para o {idioma}"

# Cria o template do prompt com mensagens do sistema e do usuário
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", "{texto}"),
])

# Gera o prompt preenchendo os parâmetros
prompt = prompt_template.invoke({"idioma": "francês", "texto": "Olá, como você está?"})
prompt  # Exibe o prompt gerado
prompt.to_messages()  # Mostra as mensagens formatadas para o modelo

[SystemMessage(content='Traduza as palavras a seguir para o francês', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Olá, como você está?', additional_kwargs={}, response_metadata={})]

## Visualizando as Mensagens do Prompt
É possível visualizar como as mensagens são formatadas antes de serem enviadas ao modelo, o que ajuda a entender a estrutura do prompt.

In [ ]:
# Importa a classe para criar prompts dinâmicos para chat
from langchain_core.prompts import ChatPromptTemplate

# Define o template do sistema para tradução parametrizada
system_template = "Traduza as palavras a seguir para o {idioma}"

# Cria o template do prompt com mensagens do sistema e do usuário
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", "{texto}"),
])

# Gera o prompt preenchendo os parâmetros
prompt = prompt_template.invoke({"idioma": "francês", "texto": "Olá, como você está?"})
prompt  # Exibe o prompt gerado
prompt.to_messages()  # Mostra as mensagens formatadas para o modelo

# Envia o prompt para o modelo e obtém a resposta traduzida
resposta = model.invoke(prompt) 
print(resposta.content)  # Exibe o texto traduzido retornado pelo modelo

Bonjour, comment vas-tu?


## Executando o Prompt e Obtendo a Tradução
Por fim, enviamos o prompt para o modelo e exibimos a resposta traduzida. Assim, é possível automatizar traduções para qualquer idioma suportado pelo modelo.